## Data Biography

<div align="center">
  <img
    src="../assets/img/test.jpeg"
    width="60%"
    style="border: 2px solid"
    alt="Description of image"
  />
  <figcaption style="font-style: italic">
    Detailed description of what the image represents
  </figcaption>
</div>
